In [1]:
!pip install transformers

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
#import plotly.express as px
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
import string
from nltk.corpus import stopwords


In [ ]:
train_df = pd.read_csv('train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('test.csv')
test_df.head()

In [18]:
#limipeza
#import nltk
#nltk.download('stopwords')

def clean(text):
    
    exclude = set(string.punctuation)
    clean_text = ''.join(ch for ch in text if ch not in exclude)
    
    exclude = set(stopwords.words('english'))
    clean_text_list = clean_text.split(' ')
    clean_text = ' '.join(ch for ch in clean_text_list if ch not in exclude)

    return clean_text

test_df['clean_text'] = test_df['text'].apply(clean)
train_df['clean_text'] = train_df['text'].apply(clean)
train_df.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds Reason earthquake May ALLAH Forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked shelter place notified off...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfires evacuation orde...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent photo Ruby Alaska smoke wildfire...


In [19]:
#Procesamiento de datos
X_train, X_test, y_train, y_test = train_test_split(train_df['clean_text'], train_df['target'], test_size=0.33,
                                                  random_state=42)
FTest = test_df['clean_text']
test_ids = test_df['id'] 

In [35]:
#Parametros
vocab_size = 30522
epochs = 6
maxlen = 60
n_words = 500
embedding_dim = 16

In [36]:
def tokenizar(textos):
    secuencia = []
    for text in textos:
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        secuencia.append(indexed_tokens)
    return secuencia


In [37]:
secuencia_train = tokenizar(X_train.values)
secuencia_test = tokenizar(X_test.values)
secuencia_FTest = tokenizar(FTest.values)

padded_train = pad_sequences(secuencia_train, maxlen = maxlen, truncating = 'post')
padded_test = pad_sequences(secuencia_test, maxlen = maxlen, truncating = 'post')
padded_FTest = pad_sequences(secuencia_FTest, maxlen = maxlen, truncating = 'post')

In [25]:
(padded_FTest[0])

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  101, 2074, 3047,
       1037, 6659, 2482, 5823,  102], dtype=int32)

In [39]:
len(secuencia_FTest[125])

43

In [38]:
#modelo Conv1D
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=maxlen),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 60, 64)            1953408   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 56, 128)           41088     
_________________________________________________________________
global_average_pooling1d_3 ( (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 2,002,817
Trainable params: 2,002,817
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(padded_train, y_train, epochs=2, validation_data=(padded_test, y_test))

Epoch 1/2
160/160 [==============================] - 96s 602ms/step - loss: 0.6149 - accuracy: 0.6435 - val_loss: 0.4685 - val_accuracy: 0.7927
Epoch 2/2
 59/160 [==========>...................] - ETA: 56s - loss: 0.3512 - accuracy: 0.8512

In [34]:
preds = model.predict_classes(padded_FTest)
#preds = pd.Series(preds)
preds = pd.Series(list((x[0] for x in preds)))
preds
df_preds = pd.concat([test_ids,preds],axis=1)
df_preds.rename(columns = {0 : 'target'}, inplace=True)
df_preds.set_index('id', inplace=True)
df_preds.to_csv('BERT-Conv1D.csv')
df_preds.head()

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


,target
id,
0,0
2,0
3,1
9,0
11,1


Tuning: 
1-  epochs = 10
    maxlen = 50
    n_words = 500
    embedding_dim = 16
Acuracy: 0.7441

2-  epochs = 6
    maxlen = 40
    n_words = 500
    embedding_dim = 16
Accuracy : 0.7656b

3-  epochs = 5
    maxlen = 40
    n_words = 300
    embedding_dim = 16
Accuracy: 0.7437
s